In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
for i in range(1,101):
    with open('simulationData\sim_'+str(i)+'.csv', 'r' ) as theFile:
        reader = csv.reader(theFile)
        headers = next(reader, None)
        actions = []
        states = []
        rewards = []
        blocks = []
        for line in reader:
            actions.append(int(line[3]))
            states.append([int(line[8]),int(line[9])])
            rewards.append(int(line[5]))
            blocks.append(int(line[0]))
        block = [
            {
                'action': np.array([[resp-1 for (id,resp) in zip(blocks,actions) if id==blockID]]),
                'state': np.array([[stimOffers for (id,stimOffers) in zip(blocks,states) if id==blockID]]),
                'reward': np.array([[outcomeRew for (id,outcomeRew) in zip(blocks,rewards) if id==blockID]]),
                'id': 'S'+str(i),

                'block': blockID
            } for blockID in np.unique(blocks)

        ]
#         block = [
#                 {
#                     'action': np.array([actions])-1,
#                     'state': np.array([states])-1,
#                     'reward': np.array([rewards]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
        if i<91:
            data['S'+str(i)] = block
        else:
            test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]]),
   'state': array([[[ 8,  9],
           [10,  9],
           [ 9,  8],
           [ 8, 10],
           [10,  8],
           [ 9,  8],
           [ 8,  9],
           [10,  9],
           [10,  8],
           [ 8,  9],
           [ 9,  8],
           [ 8,  9],
           [ 8,  9],
           [10,  9],
           [ 9,  8],
           [ 9,  8]]]),
   'reward': array([[0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]),
   'id': 'S1',
   'block': 1},
  {'action': array([[1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]]),
   'state': array([[[13, 10],
           [13, 10],
           [13, 10],
           [10, 13],
           [10, 13],
           [13, 10],
           [10, 13],
           [10,  8],
           [13, 10],
           [10,  8],
           [10,  8],
           [13, 10],
           [ 8, 10],
           [13,  8]]]),
   'reward': array([[1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]]),
   'id': 'S1',
   'block': 2},
  

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/exploreExploit_blocks/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-13 13:20:16,388 - DeepRL - DEBUG - version control: (None, None)
2021-07-13 13:20:16,390 - DeepRL - DEBUG - learning rate: 0.01
2021-07-13 13:20:16,391 - DeepRL - DEBUG - global iters: 50
2021-07-13 13:20:16,392 - DeepRL - DEBUG - training data-points: 90
2021-07-13 13:20:16,393 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-13 13:20:16,794 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/exploreExploit_blocks/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-13 13:20:23,960 - DeepRL - DEBUG - global iter =    0 total obj: 9986.8196
2021-07-13 13:20:30,760 - DeepRL - DEBUG - global iter =    1 total obj: 9980.6777
2021-07-13 13:20:37,915 - DeepRL - DEBUG - global iter =    2 total obj: 9970.7254
2021-07-13 13:20:45,067 - DeepRL - DEBUG - global iter =    3 total obj: 9964.6195
2021-07-13 13:20:52,217 - DeepRL - DEBUG - global iter =    4 total obj: 9960.0027
2021-07-13 13:20:59,418 - DeepRL - DEBUG - global iter =    5 total obj: 9956.0815
2021-07-13 13:21:06,528 - DeepRL - DEBUG - global iter =    6 total obj: 9956.6558
2021-07-13 13:21:13,584 - DeepRL - DEBUG - global iter =    7 total obj: 9948.8461
2021-07-13 13:21:20,599 - DeepRL - DEBUG - global iter =    8 total obj: 9947.5346
2021-07-13 13:21:27,658 - DeepRL - DEBUG - global iter =    9 total obj: 9945.7550
2021-07-13 13:21:34,608 - DeepRL - DEBUG - global iter =   10 total obj: 9938.5842
2021-07-13 13:21:41,591 - DeepRL - DEBUG - global iter =   11 total obj: 9938.7280
2021

INFO:tensorflow:../results/exploreExploit_blocks/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 0,  1,  1, ...,  1,  1,  0],
          [ 0,  0,  1, ...,  1, -1, -1],
          [ 0,  0,  0, ..., -1, -1, -1],
          ...,
          [ 1,  0,  1, ...,  1,  0, -1],
          [ 0,  1,  1, ...,  1,  1, -1],
          [ 1,  0,  0, ...,  1,  1, -1]]),
   'action': array([[ 1,  0,  0, ...,  0,  1,  0],
          [ 0,  1,  0, ...,  0, -1, -1],
          [ 0,  1,  1, ..., -1, -1, -1],
          ...,
          [ 1,  1,  0, ...,  0,  1, -1],
          [ 1,  1,  1, ...,  0,  0, -1],
          [ 0,  1,  0, ...,  0,  1, -1]]),
   'state': array([[[ 2,  4],
           [ 2,  4],
           [ 2,  4],
           ...,
           [ 4,  2],
           [ 2,  3],
           [ 4,  2]],
   
          [[ 4, 10],
           [10,  4],
           [10,  4],
           ...,
           [ 4,  3],
           [-1, -1],
           [-1, -1]],
   
          [[ 4,  2],
           [ 2,  4],
           [ 4,  2],
           ...,
           [-1, -1],
           [-1, -1],
           [-1, -1]]

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-13 13:26:38,062 - DeepRL - DEBUG - version control: (None, None)
2021-07-13 13:26:38,063 - DeepRL - DEBUG - learning rate: 0.01
2021-07-13 13:26:38,064 - DeepRL - DEBUG - global iters: 50
2021-07-13 13:26:38,065 - DeepRL - DEBUG - training data-points: 90
2021-07-13 13:26:38,067 - DeepRL - DEBUG - test data-points: 10
2021-07-13 13:26:38,723 - DeepRL - DEBUG - opt started...
2021-07-13 13:26:38,724 - DeepRL - DEBUG - started testing...
2021-07-13 13:26:58,850 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blocks/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-13 13:27:08,174 - DeepRL - DEBUG - global iter =    0 total obj: 9986.6312
2021-07-13 13:27:16,046 - DeepRL - DEBUG - global iter =    1 total obj: 9977.8144
2021-07-13 13:27:23,977 - DeepRL - DEBUG - global iter =    2 total obj: 9975.5637
2021-07-13 13:27:31,885 - DeepRL - DEBUG - global iter =    3 total obj: 9967.0731
2021-07-13 13:27:39,800 - DeepRL - DEBUG - global iter =    4 total obj: 9964.0288
2021-07-13 13:27:47,691 - DeepRL - DEBUG - global iter =    5 total obj: 9960.2505
2021-07-13 13:27:55,941 - DeepRL - DEBUG - global iter =    6 total obj: 9956.8376
2021-07-13 13:28:03,822 - DeepRL - DEBUG - global iter =    7 total obj: 9953.3716
2021-07-13 13:28:11,700 - DeepRL - DEBUG - global iter =    8 total obj: 9945.3216
2021-07-13 13:28:19,587 - DeepRL - DEBUG - global iter =    9 total obj: 9937.1638
2021-07-13 13:28:19,588 - DeepRL - DEBUG - started testing...
2021-07-13 13:28:44,484 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blocks/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-13 13:28:53,628 - DeepRL - DEBUG - global iter =   10 total obj: 9931.6598
2021-07-13 13:29:01,560 - DeepRL - DEBUG - global iter =   11 total obj: 9924.1334
2021-07-13 13:29:09,453 - DeepRL - DEBUG - global iter =   12 total obj: 9921.0816
2021-07-13 13:29:17,351 - DeepRL - DEBUG - global iter =   13 total obj: 9911.1528
2021-07-13 13:29:25,309 - DeepRL - DEBUG - global iter =   14 total obj: 9908.5605
2021-07-13 13:29:33,406 - DeepRL - DEBUG - global iter =   15 total obj: 9891.3029
2021-07-13 13:29:41,342 - DeepRL - DEBUG - global iter =   16 total obj: 9898.4777
2021-07-13 13:29:49,240 - DeepRL - DEBUG - global iter =   17 total obj: 9890.6383
2021-07-13 13:29:57,147 - DeepRL - DEBUG - global iter =   18 total obj: 9871.6353
2021-07-13 13:30:05,254 - DeepRL - DEBUG - global iter =   19 total obj: 9874.4755
2021-07-13 13:30:05,255 - DeepRL - DEBUG - started testing...
2021-07-13 13:30:31,429 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/exploreExploit_blocks/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-13 13:32:23,810 - DeepRL - DEBUG - global iter =   30 total obj: 9797.8138
2021-07-13 13:32:31,315 - DeepRL - DEBUG - global iter =   31 total obj: 9775.9984
2021-07-13 13:32:38,968 - DeepRL - DEBUG - global iter =   32 total obj: 9771.9409
2021-07-13 13:32:46,655 - DeepRL - DEBUG - global iter =   33 total obj: 9767.3720
2021-07-13 13:32:54,349 - DeepRL - DEBUG - global iter =   34 total obj: 9765.5495
2021-07-13 13:33:02,000 - DeepRL - DEBUG - global iter =   35 total obj: 9744.1145
2021-07-13 13:33:09,662 - DeepRL - DEBUG - global iter =   36 total obj: 9732.9183
2021-07-13 13:33:17,349 - DeepRL - DEBUG - global iter =   37 total obj: 9729.6410
2021-07-13 13:33:24,993 - DeepRL - DEBUG - global iter =   38 total obj: 9702.3408
2021-07-13 13:33:31,758 - DeepRL - DEBUG - global iter =   39 total obj: 9709.4930
2021-07-13 13:33:31,759 - DeepRL - DEBUG - started testing...
2021-07-13 13:34:04,223 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_blocks/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-13 13:34:13,294 - DeepRL - DEBUG - global iter =   40 total obj: 9709.8410
2021-07-13 13:34:20,994 - DeepRL - DEBUG - global iter =   41 total obj: 9687.0535
2021-07-13 13:34:28,740 - DeepRL - DEBUG - global iter =   42 total obj: 9728.0790
2021-07-13 13:34:36,392 - DeepRL - DEBUG - global iter =   43 total obj: 9694.3482
2021-07-13 13:34:44,033 - DeepRL - DEBUG - global iter =   44 total obj: 9661.6639
2021-07-13 13:34:51,675 - DeepRL - DEBUG - global iter =   45 total obj: 9657.8460
2021-07-13 13:34:59,303 - DeepRL - DEBUG - global iter =   46 total obj: 9661.5901
2021-07-13 13:35:06,256 - DeepRL - DEBUG - global iter =   47 total obj: 9654.9866
2021-07-13 13:35:12,640 - DeepRL - DEBUG - global iter =   48 total obj: 9658.4466
2021-07-13 13:35:18,922 - DeepRL - DEBUG - global iter =   49 total obj: 9642.5320
2021-07-13 13:35:18,923 - DeepRL - DEBUG - opt finished.
2021-07-13 13:35:18,923 - DeepRL - DEBUG - started testing...
2021-07-13 13:35:59,465 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/exploreExploit_blocks/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0]]),
  'state': array([[[1, 2],
          [1, 2],
          [1, 8],
          [8, 1],
          [1, 2],
          [2, 1],
          [2, 8],
          [1, 2],
          [8, 1],
          [1, 2],
          [2, 8],
          [1, 8],
          [8, 1],
          [8, 1],
          [2, 8],
          [2, 8]]]),
  'reward': array([[0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1]]),
  'id': 'S100',
  'block': 1},
 {'action': array([[1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0]]),
  'state': array([[[2, 9],
          [9, 2],
          [9, 2],
          [9, 2],
          [9, 2],
          [9, 2],
          [2, 9],
          [2, 9],
          [9, 2],
          [1, 9],
          [9, 2],
          [1, 2],
          [9, 1],
          [2, 1]]]),
  'reward': array([[1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]]),
  'id': 'S100',
  'block': 2},
 {'action': array([[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1]]),
  'state': array([[[1, 2],
          [2,